In [18]:
urls = ["https://base.garant.ru/70480868/53f89421bbdaf741eb2d1ecc4ddb4c33/",
        "https://base.garant.ru/70480868/f7ee959fd36b5699076b35abf4f52c5c/",
        "https://base.garant.ru/70480868/3e22e51c74db8e0b182fad67b502e640/"]

In [19]:
import requests
from bs4 import BeautifulSoup as bs

In [45]:
import re

In [43]:
content = []
for url in urls:
  page = requests.get(url)
  soup = bs(page.text, 'html.parser')
  soup.prettify()
  table = soup.find("table")
  rows = table.find_all("tr")
  content.extend([list(map(lambda p: p.text, row.find_all("p"))) for row in rows])

In [46]:
classifier = {}
for row in content:
  if not re.match(r"^(\d{2}\.){2}\d{2}$", row[0]):
    continue
  classifier[row[0]] = row[1]

In [84]:
groups = {k: {"field group name": v, "fields": {}} for k, v in classifier.items() if re.match(r"^(\d{2}\.)0{2}\.0{2}$", k)}

In [85]:
for field in classifier:
  if re.match(r"^(\d{2}\.)0{2}\.0{2}$", field):
    continue
  group_code = field[:2]
  full_group_code = f"{group_code}.00.00"
  groups[full_group_code]["fields"][field] = classifier[field]

In [86]:
groups

{'01.00.00': {'field group name': 'МАТЕМАТИКА И МЕХАНИКА',
  'fields': {'01.03.01': 'Математика',
   '01.03.02': 'Прикладная математика и информатика',
   '01.03.03': 'Механика и математическое моделирование',
   '01.03.04': 'Прикладная математика',
   '01.03.05': 'Статистика',
   '01.04.01': 'Математика',
   '01.04.02': 'Прикладная математика и информатика',
   '01.04.03': 'Механика и математическое моделирование',
   '01.04.04': 'Прикладная математика',
   '01.04.05': 'Статистика',
   '01.05.01': 'Фундаментальные математика и механика'}},
 '02.00.00': {'field group name': 'КОМПЬЮТЕРНЫЕ И ИНФОРМАЦИОННЫЕ НАУКИ',
  'fields': {'02.03.01': 'Математика и компьютерные науки',
   '02.03.02': 'Фундаментальная информатика и информационные технологии',
   '02.03.03': 'Математическое обеспечение и администрирование информационных систем',
   '02.04.01': 'Математика и компьютерные науки',
   '02.04.02': 'Фундаментальная информатика и информационные технологии',
   '02.04.03': 'Математическое обес

In [92]:
ordered_groups = {k: v for k, v in sorted(groups.items(), key=lambda item: int(item[0][:2]))}

In [93]:
ordered_groups

{'01.00.00': {'field group name': 'МАТЕМАТИКА И МЕХАНИКА',
  'fields': {'01.03.01': 'Математика',
   '01.03.02': 'Прикладная математика и информатика',
   '01.03.03': 'Механика и математическое моделирование',
   '01.03.04': 'Прикладная математика',
   '01.03.05': 'Статистика',
   '01.04.01': 'Математика',
   '01.04.02': 'Прикладная математика и информатика',
   '01.04.03': 'Механика и математическое моделирование',
   '01.04.04': 'Прикладная математика',
   '01.04.05': 'Статистика',
   '01.05.01': 'Фундаментальные математика и механика'}},
 '02.00.00': {'field group name': 'КОМПЬЮТЕРНЫЕ И ИНФОРМАЦИОННЫЕ НАУКИ',
  'fields': {'02.03.01': 'Математика и компьютерные науки',
   '02.03.02': 'Фундаментальная информатика и информационные технологии',
   '02.03.03': 'Математическое обеспечение и администрирование информационных систем',
   '02.04.01': 'Математика и компьютерные науки',
   '02.04.02': 'Фундаментальная информатика и информационные технологии',
   '02.04.03': 'Математическое обес

In [87]:
import json

# Монтируем диск
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [94]:
with open("gdrive/MyDrive/Colab Notebooks/fields_classifier.json", "w") as outfile:
  json.dump(ordered_groups, outfile)

In [117]:
query = '''insert into postgres.public."FieldOfStudy"\n values \n'''

In [118]:
for group in ordered_groups:
  item = f''' ('{group}', Null, '{groups[group]['field group name']}'),\n'''
  query += item
  for field in groups[group]['fields']:
    item = f''' ('{field}', '{group}', '{groups[group]['fields'][field]}'),\n'''
    query += item

query = query[:-2] + ";"

In [119]:
print(query)

insert into postgres.public."FieldOfStudy"
 values 
 ('01.00.00', Null, 'МАТЕМАТИКА И МЕХАНИКА'),
 ('01.03.01', '01.00.00', 'Математика'),
 ('01.03.02', '01.00.00', 'Прикладная математика и информатика'),
 ('01.03.03', '01.00.00', 'Механика и математическое моделирование'),
 ('01.03.04', '01.00.00', 'Прикладная математика'),
 ('01.03.05', '01.00.00', 'Статистика'),
 ('01.04.01', '01.00.00', 'Математика'),
 ('01.04.02', '01.00.00', 'Прикладная математика и информатика'),
 ('01.04.03', '01.00.00', 'Механика и математическое моделирование'),
 ('01.04.04', '01.00.00', 'Прикладная математика'),
 ('01.04.05', '01.00.00', 'Статистика'),
 ('01.05.01', '01.00.00', 'Фундаментальные математика и механика'),
 ('02.00.00', Null, 'КОМПЬЮТЕРНЫЕ И ИНФОРМАЦИОННЫЕ НАУКИ'),
 ('02.03.01', '02.00.00', 'Математика и компьютерные науки'),
 ('02.03.02', '02.00.00', 'Фундаментальная информатика и информационные технологии'),
 ('02.03.03', '02.00.00', 'Математическое обеспечение и администрирование информационн